Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:

- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes (aka features)

## 1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")

### TODO ANSWER GERE
## 2. Which mining techniques yielded the best results for what? How do you define "best results"?
### TODO ANSWER GERE

### 3. How useful was Principle Component Analysis? For what number of components did you get the "best results."
### TODO ANSWER GERE
PCA as alright. There was a variety of different answers. There was an even spread and no noticable
dip that made it easy to just cut off the data. It was useful for DBSCAN and K-means Clustering to aid in the
visualization

Please include tables and/or graphs to justify your statements about which are best.

### Start of The Code
Main Library Imports Below

In [8]:
import graphviz
import pandas as pd
import numpy as np
from csv import reader

# Sklearn Libraries
from sklearn import tree
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import *
from sklearn.naive_bayes import *
from sklearn.cluster import KMeans, DBSCAN
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import export_graphviz, export_text
import pydotplus

# Importing mlxtend Libraries
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Graphing Libraries
import matplotlib.pyplot as plt
import seaborn as sns

Importing the main document <br>
Setting the random seed and setting view options <br>

In [9]:
np.random.seed(0)
df =  pd.read_csv("bank-full.csv")

pd.set_option('display.max_columns', None)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.width = 0   # automatically ajust to window length

df_a = df.copy()
df_b = df.copy()

In [10]:
df_a.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Association Rule Analysis Below
### Feature Engineering
First Cleaning up the data and creating categories

In [11]:
for index, row in df_a.iterrows():
	if row['balance'] < 0:
		row['balance'] = 'inDebt'
	else:
		row['balance'] = 'notInDebt'

# Creating Categories for balance
df_a.loc[(df_a['balance'] >= 10000), 'balanceSummary'] = 'veryPositive'
df_a.loc[((df_a['balance'] < 0) & (df_a['balance'] >= -10000)), 'balanceSummary'] = 'negative'
df_a.loc[((df_a['balance'] < 10000) & (df_a['balance'] >= 0)), 'balanceSummary'] = 'positive'
df_a.loc[df_a['balance'] < -500, 'balanceSummary'] = 'veryNegative'

# Creating Categories for ages
df_a.loc[((df_a['age'] < 25) & (df_a['age'] >= 0)),  'ageBand'] = 'ageBand1'
df_a.loc[((df_a['age'] < 30) & (df_a['age'] >= 25)), 'ageBand'] = 'ageBand2'
df_a.loc[((df_a['age'] < 40) & (df_a['age'] >= 30)), 'ageBand'] = 'ageBand3'
df_a.loc[((df_a['age'] < 50) & (df_a['age'] >= 40)), 'ageBand'] = 'ageBand4'
df_a.loc[((df_a['age'] < 120) & (df_a['age'] >= 50)),'ageBand'] = 'ageBand5'

# Creating Categories for default
df_a.loc[(df_a['default'] == 'no'), 'defaultValue'] = 'defaultNo'
df_a.loc[(df_a['default'] == 'yes'), 'defaultValue'] = 'defaultYes'

# Creating Categories for Housing
df_a.loc[(df_a['housing'] == 'no'), 'housingVal'] = 'houseNo'
df_a.loc[(df_a['housing'] == 'yes'), 'housingVal'] = 'houseYes'

# Creating Categories for Loans
df_a.loc[(df_a['loan'] == 'no'), 'loanVal'] = 'loanNo'
df_a.loc[(df_a['loan'] == 'yes'), 'loanVal'] = 'loanYes'

# Dropping all values that are not strings
df_a = df_a.drop(['age', 'day', 'duration', 'balance', 'campaign', 'previous', 'default', 'housing', 'loan', 'pdays'],
                 axis=1)

# Displaying Head
df_a.head()

,job,marital,education,contact,month,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,management,married,tertiary,unknown,may,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo
1,technician,single,secondary,unknown,may,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
2,entrepreneur,married,secondary,unknown,may,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
3,blue-collar,married,unknown,unknown,may,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
4,unknown,single,unknown,unknown,may,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo


Displaying all the Unique Values

In [12]:
for col in df_a:
    print(col, ":", df_a[col].unique())

job : ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital : ['married' 'single' 'divorced']
education : ['tertiary' 'secondary' 'unknown' 'primary']
contact : ['unknown' 'cellular' 'telephone']
month : ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome : ['unknown' 'failure' 'other' 'success']
y : ['no' 'yes']
balanceSummary : ['positive' 'negative' 'veryPositive' 'veryNegative']
ageBand : ['ageBand5' 'ageBand4' 'ageBand3' 'ageBand2' 'ageBand1']
defaultValue : ['defaultNo' 'defaultYes']
housingVal : ['houseYes' 'houseNo']
loanVal : ['loanNo' 'loanYes']


Exporting the dataframe as a CSV and reimporting it to speed up run times.

In [13]:
df_a.to_csv("outfile.csv", index=False)
file = []
with open("outfile.csv", 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        file.append(row)

Creating transact 1 hot boolean encoded numpy array and displaying dataframe head

In [14]:
te = TransactionEncoder()
te_ary = te.fit(file).transform(file)
df_a_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_a_encoded.head()

,admin.,ageBand,ageBand1,ageBand2,ageBand3,ageBand4,ageBand5,apr,aug,balanceSummary,blue-collar,cellular,contact,dec,defaultNo,defaultValue,defaultYes,divorced,education,entrepreneur,failure,feb,houseNo,houseYes,housemaid,housingVal,jan,job,jul,jun,loanNo,loanVal,loanYes,management,mar,marital,married,may,month,negative,no,nov,oct,other,positive,poutcome,primary,retired,secondary,self-employed,sep,services,single,student,success,technician,telephone,tertiary,unemployed,unknown,veryNegative,veryPositive,y,yes
0,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
